Transformation class reads data from Data Source class and sends data to Data Load class(MongoDB in our case). 
It is a scalable class where we can create new methods to handle different transformation.

In our below code, we have below functions:
* apiEconomy - to handle Economy dataset transformation
* apiPollution - to handle Pollution dataset transformation

Similarly we can add new functions to handle more transformations which makes our code scalable.

In [3]:
from DataSources import Extract
from DataLoad import MongoDB
import urllib
import pandas as pd

class Transformation:
    
    def __init__(self, dataSource, dataSet):
        print('Inside')
        extractObj = Extract()
        if dataSource == 'api':
            self.data = extractObj.getAPISData(dataSet)
            funcName = dataSource+dataSet
            getattr(self, funcName)()
        elif dataSource == 'csv':
            self.data = extractObj.getCSVData(dataSet)
            funcName = dataSource+dataSet
            getattr(self, funcName)()
        else:
            self.data = extractObj.databases(dataSet)
    
    
    def apiEconomy(self):
        gdp_india = {}
        for record in self.data['records']:
            gdp={}
            gdp['GDP_in_rs_cr'] = int(record['gross_domestic_product_in_rs_cr_at_2004_05_prices'])
            gdp_india[record['financial_year']] = gdp
            gdp_india_yrs = list(gdp_india)
        for i in range(len(gdp_india_yrs)):
            if i == 0:
                pass
            else:
                key = 'GDP_Growth_' + gdp_india_yrs[i]
                gdp_india[gdp_india_yrs[i]][key] = round(((gdp_india[gdp_india_yrs[i]]['GDP_in_rs_cr'] -gdp_india[gdp_india_yrs[i-1]]['GDP_in_rs_cr'])/gdp_india[gdp_india_yrs[i-1]]['GDP_in_rs_cr'])*100,2)
        
        # connection to mongo db
        mongoDB_obj = MongoDB(urllib.parse.quote_plus('root'), urllib.parse.quote_plus('password'), 'host', 'GDP')
        # Insert Data into MongoDB
        mongoDB_obj.insert_into_db(gdp_india, 'India_GDP')
        
        
    def apiPollution(self):
        air_data = self.data['results']
        # Converting nested data into linear structure
        air_list = []
        for data in air_data:
            for measurement in data['measurements']:
                air_dict = {}
                air_dict['location'] = data['location']
                air_dict['city'] = data['city']
                air_dict['country'] = data['country']
                air_dict['parameter'] = measurement['parameter']
                air_dict['value'] = measurement['value']
                air_dict['lastUpdated'] = measurement['lastUpdated']
                air_dict['unit'] = measurement['unit']
                air_dict['sourceName'] = measurement['sourceName']
                air_list.append(air_dict)
        # Convert list of dict into pandas df
        df = pd.DataFrame(air_list, columns=air_dict.keys())
        # connection to mongo db
        mongoDB_obj = MongoDB(urllib.parse.quote_plus('root'), urllib.parse.quote_plus('password'), 'host', 'Pollution_Data')
        # Insert Data into MongoDB
        mongoDB_obj.insert_into_db(df, 'Air_Quality_India')

    def csvCryptoMarkets(self):
        print('Please Code me !!!!')
    


You can think of __name__ == '__main__' as void main, when Python interpret this file, it sets __name__  to "__main__"
and starts executing the code from there only.

In [4]:
if __name__ == '__main__':
    dataSource = input("Please Select the DataSource i.e 'API'/'CSV'/'Database': ").lower()
    print(dataSource)
    dataSet = input('Please select the Data set for Transformation: ').title()
    print(dataSet)
    trans_obj = Transformation(dataSource, dataSet)
    
    

Please Select the DataSource i.e 'API'/'CSV'/'Database': api
api
Please select the Data set for Transformation: pollution
Pollution
Inside
MongoDB Connection Successful. CHEERS!!!
Data Inserted Successfully
Connection Closed!!!
